In [5]:
!pip3 install opencv-python --break-system-packages
!pip3 install pyqt5 --break-system-packages
!pip3 install scipy --break-system-packages

In [ ]:
# import pycuda.driver as cuda
# import pycuda.autoinit
# from pycuda.compiler import SourceModule
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import cv2
import time

from config import *
from util import *
from leapfrog2D import *

ENGINE = 'qt5'

if ENGINE == 'qt5':
    %matplotlib qt5
else:
    %matplotlib widget

ModuleNotFoundError: No module named 'pycuda'

In [ ]:
#S = configSimulation('laser pulse', anecroic=True)
#S = configSimulation('single lens')
#S = configSimulation('single lens plane')
#S = configSimulation('yagi_antenna')
#S = configSimulation('antireflex')
#S = configSimulation('opticalfiber')
#S = configSimulation('optical ckt')
#S = configSimulation('dual optcal fiber')
#S = configSimulation('refraction')  # mega demo!

# homework simulations
#S = configSimulation('wall right')
#S = configSimulation('wall left')
#S = configSimulation('wall above')
#S = configSimulation('wall below')
#S = configSimulation('room')
#S = configSimulation('closed room')
#S = configSimulation('room with lens')
#S = configSimulation('room with receiver')
S = configSimulation('flat room with receiver')

# config variables
Nt = S['Nt']
dt = S['dt']
Nx = S['Nx']
Ny = S['Ny']

# initial field values
Hx = np.zeros((S['Ny'],S['Nx']))
Hy = np.zeros((S['Ny'],S['Nx']))
Ez = np.zeros((S['Ny'],S['Nx']))

# medium
s = S['s']
er = S['er']
ur = S['ur']
wire = S['wire']
dt = S['dt']

# sources
sources_parameters = S['sources_parameters']
sources_names = S['sources_names']

# FD-TD constants (normalization discretization steps, etc)
[c1, c2, c3, c4] = leapFrog2D_constants(S['er'], S['ur'], S['s'], S['dt']);

parameters = {}
parameters['c1'] = c1;
parameters['c2'] = c2;
parameters['c3'] = c3;
parameters['c4'] = c4;
parameters['dx'] = S['dx']
parameters['dy'] = S['dy']
parameters['wire'] = wire

# colormaps
[M1, M2, M3, M4] = customColormaps();
Imaterial = composeImage(s, wire, er, M2, M3, M4)

# Drawing stuff

fig, ax = plt.subplots(figsize=(10, 8))
ax_img = ax.imshow(Imaterial)
ax.set_aspect('equal')
ax.set_axis_off()

fourcc = cv2.VideoWriter_fourcc(*"X264")
video_out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (Nx,Ny))

# ---------------------------------------------------------
# CONFIGURAÇÃO DO RECEIVER 
# ---------------------------------------------------------
receiver_data = [] # Lista para guardar o sinal capturado
rx_pixel = 0
ry_pixel = 0

has_receiver = 'receivers' in S and len(S['receivers']) > 0

if has_receiver:
    # Pega o primeiro receiver da lista
    rec = S['receivers'][0] 
    
    # Converte de 0.0-1.0 para Pixels (Inteiros)
    rx_pixel = int(rec['cx'] * Nx)
    ry_pixel = int(rec['cy'] * Ny)
    
    # Garante que está dentro da tela
    rx_pixel = np.clip(rx_pixel, 0, Nx-1)
    ry_pixel = np.clip(ry_pixel, 0, Ny-1)
    
    print(f"Receiver posicionado em: X={rx_pixel}, Y={ry_pixel}")

# ---------------------------------------------------------


# main time loop 
t = 0
n = 1

while n<Nt:
    n = n+1;
    t = t + dt

    # inject sources
    for source_parameter, source_name in zip(sources_parameters, sources_names):
        source_parameter['time'] = t;
        Ez += generateSource(Nx, Ny, source_name,source_parameter)
    
    
    # Perform Mathemagic
    leapFrog2D(Ez, Hy, Hx, parameters)

    # -----------------------------------------------------
    # CAPTURA DO SINAL PELO RECEIVER
    # -----------------------------------------------------
    
    if has_receiver:
        # Salva o valor do Campo Elétrico (Ez) naquele ponto exato
        val = Ez[ry_pixel, rx_pixel]
        receiver_data.append(val)
    # -----------------------------------------------------
    # draw stuff
    if n%2==0:
        I = addEzToComposite(Imaterial, Ez/5.0, M1)
        # -------------------------------------------------
        # DESENHAR O 'X' DO RECEIVER NA IMAGEM
        # -------------------------------------------------
        if has_receiver:
            
            # Tamanho do marcador
            size = 5 
            
            # Cor vermelha [255, 0, 0]
            
            y_min = max(0, ry_pixel - size)
            y_max = min(Ny, ry_pixel + size)
            x_min = max(0, rx_pixel - size)
            x_max = min(Nx, rx_pixel + size)

            # Desenha um quadrado vermelho onde está o receiver para vê-lo
            I[y_min:y_max, x_min:x_max, 0] = 255 # R
            I[y_min:y_max, x_min:x_max, 1] = 0   # G
            I[y_min:y_max, x_min:x_max, 2] = 0   # B

        if ENGINE == 'qt5':
            ax_img.set_data(I)
            plt.pause(0.0001)
        else:
            ax.clear()
            ax.imshow(I)
            clear_output(wait=True)
            display(fig)


    video_out.write(I[:,:,[2, 1, 0]])


[configSimulation] Unknown configuration name: flat room with receiver
Receiver posicionado em: X=560, Y=208


KeyboardInterrupt: 

In [ ]:
video_out.release()

In [3]:
plt.imshow(S['s'])
plt.show()

In [4]:
# Plota o sinal capturado
if has_receiver:
    plt.figure(figsize=(10,4))
    plt.plot(receiver_data)
    plt.title(f"Sinal recebido na posição X={rx_pixel}, Y={ry_pixel}")
    plt.xlabel("Tempo (steps)")
    plt.ylabel("Amplitude (Ez)")
    plt.grid(True)
    plt.show()
else:
    print("Nenhum receiver configurado.")